In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [4]:
import keras
from keras.models import load_model
from keras.losses import binary_crossentropy, mean_squared_error, mean_squared_error

Using TensorFlow backend.
/Users/sergmiller/anaconda3/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [91]:
import building, training, models, callbacks, wonder

In [7]:
from importlib import reload

In [93]:
reload(building)
reload(training)
reload(models)
reload(callbacks)
reload(wonder)

<module 'wonder' from '/Users/sergmiller/Documents/code/python/alissa/wonder.py'>

In [189]:
model = models.model((40, 300))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context1 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
context2 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
context3 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
context4 (InputLayer)           (None, 40, 300)      0                                            
__________________________________________________________________________________________________
model_159 

In [9]:
emb = building.make_emb('datasets/ru.csv')

In [10]:
train = pd.read_csv('datasets/sents/train_sents').fillna('')
val = pd.read_csv('datasets/sents/val_sents').fillna('')
boost = pd.read_csv('datasets/sents/test_sents').fillna('')
public = pd.read_csv('datasets/public_df', index_col='Unnamed: 0').fillna('')

In [11]:
train.head()

,0,1,2,3,4,5,6,7
0,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,0,"да , в ванной .",good,0.942226
1,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,1,я не грязный .,bad,0.812665
2,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,2,"так , будь готова .",good,0.488691
3,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,3,на тот холм .,bad,0.996344
4,94846308296151,"я пойду , э , освежусь .","сделай массажный душ , джамбо .",хмм ?,4,ты идешь внутрь .,bad,0.506997


In [12]:
public.head()

,0,1,2,3,4,5
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,4,"разумеется , нет ."


In [13]:
val.shape

(7876, 8)

In [206]:
def run_model(train, val, emb, epochs=20, batch_size=64, sample_len=10000, load_file=None, save_file=None):
    if load_file is None:
        model = models.model((40, 300), eps=1e-5)
    else:
        model = load_model(load_file)

        model.compile(
            loss={'class_out': binary_crossentropy, 
                  'auto3' : mean_squared_error, 
                  'auto4': mean_squared_error},
            loss_weights = {'class_out': 1, 
                  'auto3' : 0, 
                  'auto4': 0},
            optimizer='rmsprop',
        )
    
    filepath = "weights/weights.{epoch:02d}-{val_loss:.3f}.hdf5"
    
    y_train = (train['6'] == 'good').values
    y_val = (val['6'] == 'good').values
    w_train = train['7']
    
    sample_train = building.shuffle_by_groups(train, '0', random_state=0)[:sample_len]

    model.fit_generator(training.flow(train, emb, batch_size, y_train, w_train),
                        steps_per_epoch=train.shape[0] / batch_size, 
                        epochs=epochs, 
                        shuffle=True,
                        callbacks=[callbacks.TelegramCallback(), 
                                callbacks.RocCallback(sample_train, val, emb),
                                callbacks.NDCGCallback(sample_train, val, emb),
                                keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9,
                              patience=3, min_lr=1e-5),
                                keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, 
                                    save_best_only=False, save_weights_only=False, mode='auto', period=1)
                                  ],
                        validation_data=training.flow(val, emb, batch_size, y_train),
                        validation_steps=val.shape[0] / batch_size,
                       )

    if save_file is not None:
        model.save(save_file)

In [207]:
run_model(train, val, emb, load_file='weights/weights.06-2.292.hdf5')

Epoch 1/20
1108/1107 [==============================] - 339s 306ms/step - loss: 0.5368 - class_out_loss: 0.5137 - auto3_loss: 0.9561 - auto4_loss: 0.8353 - val_loss: 0.9785 - val_class_out_loss: 0.9547 - val_auto3_loss: 0.9770 - val_auto4_loss: 0.8560
Epoch 2/20
1108/1107 [==============================] - 302s 273ms/step - loss: 0.5136 - class_out_loss: 0.4884 - auto3_loss: 0.9654 - auto4_loss: 0.8447 - val_loss: 0.9847 - val_class_out_loss: 0.9581 - val_auto3_loss: 0.9757 - val_auto4_loss: 0.8546
Epoch 3/20
1108/1107 [==============================] - 251s 227ms/step - loss: 4.7504 - class_out_loss: 4.7208 - auto3_loss: 0.9655 - auto4_loss: 0.8446 - val_loss: 8.5274 - val_class_out_loss: 8.5023 - val_auto3_loss: 0.9771 - val_auto4_loss: 0.8553
Epoch 4/20
1108/1107 [==============================] - 304s 274ms/step - loss: 6.3211 - class_out_loss: 6.2982 - auto3_loss: 0.9655 - auto4_loss: 0.8447 - val_loss: 7.7430 - val_class_out_loss: 7.7206 - val_auto3_loss: 0.9776 - val_auto4_loss:

In [96]:
model = load_model('weights/golden/weights.02-0.985.hdf5')

model.compile(
    loss={'class_out': binary_crossentropy, 
          'auto3' : mean_squared_error, 
          'auto4': mean_squared_error},
    loss_weights = {'class_out': 1, 
          'auto3' : 0, 
          'auto4': 0},
    optimizer='rmsprop',
)

In [97]:
training.predict(model, boost, emb, use_y=True)

ndcg_mean: 0.950366904302501, ndcg_std: 0.06818249676731028


array([[0.6410377 ],
       [0.97201127],
       [0.58125263],
       ...,
       [0.61131245],
       [0.74294466],
       [0.7219546 ]], dtype=float32)

у недоученной модели 0.944 +- 0.070

In [98]:
public_pred = training.predict(model, public, emb, use_y=False)

In [99]:
sub = wonder.create_sub(public, public_pred, 'submissions/sub_only_with_tuned_model0.csv')